In [1]:
import pandas as pd
import sqlite3, matplotlib
from sqlalchemy import create_engine
import re

Read data from CSV

In [2]:
# read the Park Data
df_pr = pd.read_csv('parks.csv', encoding='utf-8')
# read the Species Data
df_sp = pd.read_csv('species.csv', encoding='utf-8')

/Users/minglei/anaconda3/envs/PythonData/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df_pr.head()

,Park Code,Park Name,State,Acres,Latitude,Longitude
0,ACAD,Acadia National Park,ME,47390,44.35,-68.21
1,ARCH,Arches National Park,UT,76519,38.68,-109.57
2,BADL,Badlands National Park,SD,242756,43.75,-102.50
3,BIBE,Big Bend National Park,TX,801163,29.25,-103.25
4,BISC,Biscayne National Park,FL,172924,25.65,-80.08


In [4]:
df_sp.head(100)

,Species ID,Park Name,Category,Order,Family,Scientific Name,Common Names,Record Status,Occurrence,Nativeness,Abundance,Seasonality,Conservation Status,Unnamed: 13
0,ACAD-1000,Acadia National Park,Mammal,Artiodactyla,Cervidae,Alces alces,Moose,Approved,Present,Native,Rare,Resident,NaN,NaN
1,ACAD-1001,Acadia National Park,Mammal,Artiodactyla,Cervidae,Odocoileus virginianus,"Northern White-Tailed Deer, Virginia Deer, Whi...",Approved,Present,Native,Abundant,NaN,NaN,NaN
2,ACAD-1002,Acadia National Park,Mammal,Carnivora,Canidae,Canis latrans,"Coyote, Eastern Coyote",Approved,Present,Not Native,Common,NaN,Species of Concern,NaN
3,ACAD-1003,Acadia National Park,Mammal,Carnivora,Canidae,Canis lupus,"Eastern Timber Wolf, Gray Wolf, Timber Wolf",Approved,Not Confirmed,Native,NaN,NaN,Endangered,NaN
4,ACAD-1004,Acadia National Park,Mammal,Carnivora,Canidae,Vulpes vulpes,"Black Fox, Cross Fox, Eastern Red Fox, Fox, Re...",Approved,Present,Unknown,Common,Breeder,NaN,NaN
5,ACAD-1005,Acadia National Park,Mammal,Carnivora,Felidae,Lynx canadensis,Canada Lynx,Approved,Not Confirmed,Native,NaN,NaN,NaN,NaN
6,ACAD-1006,Acadia National Park,Mammal,Carnivora,Felidae,Lynx rufus,"Bay Lynx, Bobcat, Red Lynx, Wild Cat",Approved,Not Present (Historical Report),Not Native,NaN,Vagrant,NaN,NaN
7,ACAD-1007,Acadia National Park,Mammal,Carnivora,Mephitidae,Mephitis mephitis,"Eastern Skunk, Polecat, Striped Skunk",Approved,Not Present (Historical Report),Native,NaN,NaN,NaN,NaN
8,ACAD-1008,Acadia National Park,Mammal,Carnivora,Mustelidae,Lutra canadensis,"Otter, River Otter",Approved,Present,Native,Common,Breeder,NaN,NaN
9,ACAD-1009,Acadia National Park,Mammal,Carnivora,Mustelidae,Martes pennanti,"Blackcat, Fisher, Pekan",Approved,Present,Native,Rare,NaN,NaN,NaN


In [5]:
df_pr.columns

Index(['Park Code', 'Park Name', 'State', 'Acres', 'Latitude', 'Longitude'], dtype='object')

In [6]:
df_pr.rename(columns={'Park Code':'park_code','Park Name':'park_name','State':'state','Acres':'acres','Latitude':'latitude','Longitude':'longitude'}, inplace=True)

In [7]:
df_pr.columns

Index(['park_code', 'park_name', 'state', 'acres', 'latitude', 'longitude'], dtype='object')

In [8]:
df_sp.columns

Index(['Species ID', 'Park Name', 'Category', 'Order', 'Family',
       'Scientific Name', 'Common Names', 'Record Status', 'Occurrence',
       'Nativeness', 'Abundance', 'Seasonality', 'Conservation Status',
       'Unnamed: 13'],
      dtype='object')

In [9]:
df_sp.drop(['Unnamed: 13'], axis=1, inplace=True)

In [10]:
df_sp.rename(columns={'Species ID':'species_ID','Park Name':'park_name','Category':'category','Order':'order','Family':'family','Scientific Name':'scientific_name','Common Names':'common_names','Record Status':'record_status','Occurence':'occurence','Nativeness':'nativeness','Abundance':'abundance','Seasonability':'seasonability','Conservation Status':'conservation_status'}, inplace=True)

In [11]:
df_sp.drop(['species_ID','scientific_name','record_status','Occurrence',
       'nativeness', 'abundance', 'Seasonality', 'conservation_status'], axis=1, inplace=True)

In [12]:
df_sp.columns

Index(['park_name', 'category', 'order', 'family', 'common_names'], dtype='object')

In [13]:
df_sp.head(400)

,park_name,category,order,family,common_names
0,Acadia National Park,Mammal,Artiodactyla,Cervidae,Moose
1,Acadia National Park,Mammal,Artiodactyla,Cervidae,"Northern White-Tailed Deer, Virginia Deer, Whi..."
2,Acadia National Park,Mammal,Carnivora,Canidae,"Coyote, Eastern Coyote"
3,Acadia National Park,Mammal,Carnivora,Canidae,"Eastern Timber Wolf, Gray Wolf, Timber Wolf"
4,Acadia National Park,Mammal,Carnivora,Canidae,"Black Fox, Cross Fox, Eastern Red Fox, Fox, Re..."
5,Acadia National Park,Mammal,Carnivora,Felidae,Canada Lynx
6,Acadia National Park,Mammal,Carnivora,Felidae,"Bay Lynx, Bobcat, Red Lynx, Wild Cat"
7,Acadia National Park,Mammal,Carnivora,Mephitidae,"Eastern Skunk, Polecat, Striped Skunk"
8,Acadia National Park,Mammal,Carnivora,Mustelidae,"Otter, River Otter"
9,Acadia National Park,Mammal,Carnivora,Mustelidae,"Blackcat, Fisher, Pekan"


In [14]:
df_pr= df_pr.iloc[:,0:6]
df_pr.head()

,park_code,park_name,state,acres,latitude,longitude
0,ACAD,Acadia National Park,ME,47390,44.35,-68.21
1,ARCH,Arches National Park,UT,76519,38.68,-109.57
2,BADL,Badlands National Park,SD,242756,43.75,-102.50
3,BIBE,Big Bend National Park,TX,801163,29.25,-103.25
4,BISC,Biscayne National Park,FL,172924,25.65,-80.08


Performing linear interpolation at missing datapoints

In [15]:
missing_data_rows = df_pr[df_pr.isnull().any(1)]
missing_data_rows

,park_code,park_name,state,acres,latitude,longitude


In [16]:
# check if the missing datapoints still exist 
check_missing_data_rows = df_pr[df_pr.isnull().any(1)]
check_missing_data_rows

,park_code,park_name,state,acres,latitude,longitude


Save to the sqlite database

In [17]:
engine = create_engine('sqlite:///parks_and_species.db', echo=True)

In [18]:
# save the parks data into sqlite
df_pr.to_sql('parks', con=engine, if_exists='replace')

2019-08-23 11:40:50,006 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2019-08-23 11:40:50,008 INFO sqlalchemy.engine.base.Engine ()
2019-08-23 11:40:50,014 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2019-08-23 11:40:50,020 INFO sqlalchemy.engine.base.Engine ()
2019-08-23 11:40:50,023 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("parks")
2019-08-23 11:40:50,025 INFO sqlalchemy.engine.base.Engine ()
2019-08-23 11:40:50,031 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("parks")
2019-08-23 11:40:50,032 INFO sqlalchemy.engine.base.Engine ()
2019-08-23 11:40:50,037 INFO sqlalchemy.engine.base.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2019-08-23 11:40:50,040 INFO sqlalchemy.engine.base.Engine ()
2019-08-23 11:40:50,051 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("parks")
2019-08-23 11:40:50,055 INFO sqlalchemy.engine.base.Engine ()
2019-08-2

In [19]:
# save the divorce data into sqlite
df_sp.to_sql('species', con=engine, if_exists='replace')

2019-08-23 11:40:50,406 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("species")
2019-08-23 11:40:50,407 INFO sqlalchemy.engine.base.Engine ()
2019-08-23 11:40:50,410 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("species")
2019-08-23 11:40:50,413 INFO sqlalchemy.engine.base.Engine ()
2019-08-23 11:40:50,415 INFO sqlalchemy.engine.base.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2019-08-23 11:40:50,416 INFO sqlalchemy.engine.base.Engine ()
2019-08-23 11:40:50,418 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("species")
2019-08-23 11:40:50,419 INFO sqlalchemy.engine.base.Engine ()
2019-08-23 11:40:50,424 INFO sqlalchemy.engine.base.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = 'species' AND type = 'table'
2019-08-23 11:40:50,425 INFO sqlalchemy.engine.base.Engine ()
2019-08-23 11:40:50,427 INFO sqlalchemy.engine.base.Engine PRAGMA foreign_key_list("species")
2019-08-23 11:40

In [20]:
engine.execute("SELECT * FROM parks").fetchall()

2019-08-23 11:40:52,481 INFO sqlalchemy.engine.base.Engine SELECT * FROM parks
2019-08-23 11:40:52,482 INFO sqlalchemy.engine.base.Engine ()


[(0, 'ACAD', 'Acadia National Park', 'ME', 47390, 44.35, -68.21),
 (1, 'ARCH', 'Arches National Park', 'UT', 76519, 38.68, -109.57),
 (2, 'BADL', 'Badlands National Park', 'SD', 242756, 43.75, -102.5),
 (3, 'BIBE', 'Big Bend National Park', 'TX', 801163, 29.25, -103.25),
 (4, 'BISC', 'Biscayne National Park', 'FL', 172924, 25.65, -80.08),
 (5, 'BLCA', 'Black Canyon of the Gunnison National Park', 'CO', 32950, 38.57, -107.72),
 (6, 'BRCA', 'Bryce Canyon National Park', 'UT', 35835, 37.57, -112.18),
 (7, 'CANY', 'Canyonlands National Park', 'UT', 337598, 38.2, -109.93),
 (8, 'CARE', 'Capitol Reef National Park', 'UT', 241904, 38.2, -111.17),
 (9, 'CAVE', 'Carlsbad Caverns National Park', 'NM', 46766, 32.17, -104.44),
 (10, 'CHIS', 'Channel Islands National Park', 'CA', 249561, 34.01, -119.42),
 (11, 'CONG', 'Congaree National Park', 'SC', 26546, 33.78, -80.78),
 (12, 'CRLA', 'Crater Lake National Park', 'OR', 183224, 42.94, -122.1),
 (13, 'CUVA', 'Cuyahoga Valley National Park', 'OH', 32

In [21]:
engine.execute("SELECT * FROM species").fetchall()

2019-08-23 11:40:52,501 INFO sqlalchemy.engine.base.Engine SELECT * FROM species
2019-08-23 11:40:52,503 INFO sqlalchemy.engine.base.Engine ()


[(0, 'Acadia National Park', 'Mammal', 'Artiodactyla', 'Cervidae', 'Moose'),
 (1, 'Acadia National Park', 'Mammal', 'Artiodactyla', 'Cervidae', 'Northern White-Tailed Deer, Virginia Deer, White-Tailed Deer'),
 (2, 'Acadia National Park', 'Mammal', 'Carnivora', 'Canidae', 'Coyote, Eastern Coyote'),
 (3, 'Acadia National Park', 'Mammal', 'Carnivora', 'Canidae', 'Eastern Timber Wolf, Gray Wolf, Timber Wolf'),
 (4, 'Acadia National Park', 'Mammal', 'Carnivora', 'Canidae', 'Black Fox, Cross Fox, Eastern Red Fox, Fox, Red Fox, Silver Fox'),
 (5, 'Acadia National Park', 'Mammal', 'Carnivora', 'Felidae', 'Canada Lynx'),
 (6, 'Acadia National Park', 'Mammal', 'Carnivora', 'Felidae', 'Bay Lynx, Bobcat, Red Lynx, Wild Cat'),
 (7, 'Acadia National Park', 'Mammal', 'Carnivora', 'Mephitidae', 'Eastern Skunk, Polecat, Striped Skunk'),
 (8, 'Acadia National Park', 'Mammal', 'Carnivora', 'Mustelidae', 'Otter, River Otter'),
 (9, 'Acadia National Park', 'Mammal', 'Carnivora', 'Mustelidae', 'Blackcat, Fi

In [22]:
engine.execute("SELECT * FROM species where park_name = 'Acadia National Park'").fetchall()

2019-08-23 11:40:52,930 INFO sqlalchemy.engine.base.Engine SELECT * FROM species where park_name = 'Acadia National Park'
2019-08-23 11:40:52,931 INFO sqlalchemy.engine.base.Engine ()


[(0, 'Acadia National Park', 'Mammal', 'Artiodactyla', 'Cervidae', 'Moose'),
 (1, 'Acadia National Park', 'Mammal', 'Artiodactyla', 'Cervidae', 'Northern White-Tailed Deer, Virginia Deer, White-Tailed Deer'),
 (2, 'Acadia National Park', 'Mammal', 'Carnivora', 'Canidae', 'Coyote, Eastern Coyote'),
 (3, 'Acadia National Park', 'Mammal', 'Carnivora', 'Canidae', 'Eastern Timber Wolf, Gray Wolf, Timber Wolf'),
 (4, 'Acadia National Park', 'Mammal', 'Carnivora', 'Canidae', 'Black Fox, Cross Fox, Eastern Red Fox, Fox, Red Fox, Silver Fox'),
 (5, 'Acadia National Park', 'Mammal', 'Carnivora', 'Felidae', 'Canada Lynx'),
 (6, 'Acadia National Park', 'Mammal', 'Carnivora', 'Felidae', 'Bay Lynx, Bobcat, Red Lynx, Wild Cat'),
 (7, 'Acadia National Park', 'Mammal', 'Carnivora', 'Mephitidae', 'Eastern Skunk, Polecat, Striped Skunk'),
 (8, 'Acadia National Park', 'Mammal', 'Carnivora', 'Mustelidae', 'Otter, River Otter'),
 (9, 'Acadia National Park', 'Mammal', 'Carnivora', 'Mustelidae', 'Blackcat, Fi

In [26]:
df_pr.to_json(orient='records')

'[{"park_code":"ACAD","park_name":"Acadia National Park","state":"ME","acres":47390,"latitude":44.35,"longitude":-68.21},{"park_code":"ARCH","park_name":"Arches National Park","state":"UT","acres":76519,"latitude":38.68,"longitude":-109.57},{"park_code":"BADL","park_name":"Badlands National Park","state":"SD","acres":242756,"latitude":43.75,"longitude":-102.5},{"park_code":"BIBE","park_name":"Big Bend National Park","state":"TX","acres":801163,"latitude":29.25,"longitude":-103.25},{"park_code":"BISC","park_name":"Biscayne National Park","state":"FL","acres":172924,"latitude":25.65,"longitude":-80.08},{"park_code":"BLCA","park_name":"Black Canyon of the Gunnison National Park","state":"CO","acres":32950,"latitude":38.57,"longitude":-107.72},{"park_code":"BRCA","park_name":"Bryce Canyon National Park","state":"UT","acres":35835,"latitude":37.57,"longitude":-112.18},{"park_code":"CANY","park_name":"Canyonlands National Park","state":"UT","acres":337598,"latitude":38.2,"longitude":-109.93}